In [ ]:
import json
with open('../subj_corresp.json', 'r') as f:
  subj_corresp = json.load(f)

In [ ]:
import json
with open('../from_Hilbert/entries_full.json', 'r') as f:
  entries_full = json.load(f)

import json
with open('../from_Hilbert/entries_trem_info.json', 'r') as f:
  entries_trem_info = json.load(f)

new_IDs_with_trem = []
oldnames_with_trem = []
for key,en in entries_trem_info.items():
    istrem = en['rest tremor left med off'] or en['rest tremor right med off    ']
    if istrem:
        new_IDs_with_trem += [entries_full[key]['new_ID']]
        oldnames_with_trem += [key]


In [ ]:
rawdir = '/home/demitau/hhuc_gpfs/project/todorov/ai200/rawdata_copy'
data_dir_input = rawdir

In [ ]:
len(new_IDs_with_trem)

In [ ]:
inds = np.arange(40)  # 242 Gb
inds = np.arange(40,48)  #75
inds = np.arange(0,48)  #75
new_IDs_to_to_process=list(sorted(new_IDs_with_trem)) 
new_IDs_to_to_process = np.array(new_IDs_to_to_process)[ inds]
#
new_IDs_to_to_process = ['S014']
fnames_all = list(os.listdir(rawdir))
fnames_per_new_ID = {}
info_times = {}

fname_full_list = []
fname_list = []
ERROR=False
#for new_ID in patients_ids:
for new_ID in new_IDs_to_to_process:
#for oldname,d in entries_full.items():
    fnames = [ fn for fn in fnames_all if fn.startswith(new_ID) ]
    print(f'for new_ID={new_ID} found {len(fnames)} filenames')
    if len(fnames):
        fnames_per_new_ID[new_ID] = fnames
        print(fnames)
    else:
        continue

    info_times[new_ID] = {}

    for fname in fnames:
        fname_full = os.path.join(data_dir_input,fname)        
        if not os.path.exists(fname_full):
            print('Warning: path does not exist!, skip! {}'.format(fname_full))
            continue
        
        fname_list += [fname]
        fname_full_list += [fname_full]

total_size = 0
for fname_full in fname_full_list:
    st = os.stat(fname_full)
    # in GB
    sz = st.st_size / (1024  * 1024 * 1024)
    total_size += sz
print(total_size)

In [ ]:
data_dir_input

In [ ]:
#remote_dir_ssh = 'hhuc:rawdata_copy'
remote_dir_ssh = f'hhuc:/home/todorov/rawdata'
import shutil
s = f'rsync -rtvzn --progress '
#s += f'{data_dir_input}' + r'/{'
s += f' --partial --append-verify {remote_dir_ssh}' + r'/{'
for fn in fname_list:    
    s += fn + ','
#dest = ' /home/demitau/data/raws_from_Hilbert'
#dest = ' /media/demitau/85121478-c7b6-4d0b-a865-d2885e75bc28/raws_from_Hilbert/'
dest = r' $DEST'
s = s[:-1] + r'}' + dest
#    shutil.copyfile(src, dst)    
#shutil.copyfile(src, dst)
print(s)

In [ ]:
terminator -e "until ssh jusuf_3 -t 'tmux a -t stat'; do sleep 1; done;"



In [ ]:
#!jt -t chesterish

In [ ]:
from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('onedork') # chesterish, monokai

In [ ]:
entries_trem_info

In [ ]:
skip_existing_EMG = False
rectconv_filter_again = False
#14
#S037_PD
from os.path import join as pjoin
import utils_preprocess as upre
# TODO: put right data directories
data_dir_input = ''
data_dir_output = ''
rawname = 'S037_PD'
for rawname_ in rawnames:

#for subjind in subjinds:
#    sis = '{:02d}'.format(subjind)
#    for medstate in medstates:
#        for task in tasks:

    #fnames_noext = ['S{}_off_{}'.format(sis,task), 'S{}_on_{}'.format(sis,task)]
    #fnames_noext = ['S01_off_hold', 'S01_on_hold']
    #fnames_noext = ['S01_off_move', 'S01_on_move']
    #fnames_noext = ['S02_off_move', 'S02_on_move']
    #fname_noext = 'S{}_{}_{}'.format(sis,medstate,task)

    fname_noext = rawname_
    fname = fname_noext + '.mat'
    print('Reading {}'.format(fname) )

    addStr = ''
    print('--- Starting reading big 2kHz file!')
    fname_full = os.path.join(data_dir_input,fname)

    if not os.path.exists(fname_full):
        print('Warning: path does not exist!, skip! {}'.format(fname_full))
        continue

    f = upre.read_raw_fieldtrip(fname_full, None)
    rectconvraw = upre.extractEMGData(f,fname_noext, skip_if_exist = skip_existing_EMG, 
                        save_dir = data_dir_output)  #saves emg_rectconv

#     mod_info, infos = upre.readInfo(fname_noext, f)
#     f.info = mod_info

#     raw_lfp = upre.saveLFP(fname_noext, skip_if_exist =
#                            skip_existing_LFP,sfreq=freqResample, raw_FT=f,n_jobs=n_jobs)
#     raw_lfp_highres = upre.saveLFP(fname_noext, skip_if_exist =
#                                    skip_existing_LFP,sfreq=freqResample_high,
#                                    raw_FT=f,n_jobs=n_jobs )

    rectconvraw.apply_function( lambda x: x / np.quantile(x,0.75) )
    hilbraw = rectconvraw.copy()
    if rectconv_filter_again:
        hilbraw.filter(l_freq=2,h_freq=10)
    hilbraw.apply_hilbert()

    # smoothness of hilb_freq depends heavilly on the band we use for filtering hilbraw
    hilb_amp = hilbraw.copy()
    hilb_amp.apply_function(np.abs)
    
    rectconv_env_fname_full = os.path.join(data_dir_output, '{}_emg_rectconv_envelope.fif'.format(rawname_) )
    if not (skip_existing_EMG and os.path.exists(rectconv_env_fname_full) ):
        print('EMG hilbert amp raw saved to ',rectconv_env_fname_full)
        hilb_amp.save(rectconv_env_fname_full, overwrite=1)